## <center>Projet RP</center>

In [14]:
import os
import random

def get_words(file="dico.txt"):
    words = []
    file_path = os.getcwd() + "/" + file
    with open(file_path, "r") as f:
        for line in f.readlines():
            words.append(line[:-1])

    return words


In [12]:
def get_secret_word(words):
    return words[random.randint(0, len(words))]

In [17]:
words = get_words()
secret_word = get_secret_word(words)

print(secret_word)

sinusitis


In [1]:
def compare_string(word, secret_word):
    right, wrong = 0, 0
    
    for w1, w2 in zip(word, secret_word):
        if w1 == w2:
            right += 1
        else:
            wrong += 1

    return right, wrong

In [ ]:
def backtracking(n, secret_word, words=words):
    